In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd

file_path = '/content/gdrive/MyDrive/Books.json'

df = pd.read_json(file_path, lines=True)

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1E5ZR1Z4OQJG,1495329321,"Pure Jonel ""Pure Jonel""","[0, 0]",Da Silva takes the divine by storm with this u...,4,An amazing first novel,1396137600,"03 30, 2014"
1,A30PZPI6FPH0A7,0399157565,Jackmollie,"[0, 0]",For me personally it's the most disappointing ...,2,disappointed,1400112000,"05 15, 2014"
2,A1GQ2UI5BKCCRD,0984528105,Gail Hodges,"[0, 0]","Very simple book, but leaves you feeling good....",4,Good book,1401235200,"05 28, 2014"
3,A2DF4LQQI6KSQ2,0804139024,Olga,"[0, 0]",I read a library copy of this exceptionally we...,5,Science Fiction at its best!,1396483200,"04 3, 2014"
4,A1UAMAWY966P2,0765317583,Nadyne M Ichimura,"[0, 0]",With the government knowing this could happen ...,5,Excellent story,1397001600,"04 9, 2014"


In [ ]:
df.keys()

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [ ]:
from collections import Counter

counter = Counter(df['overall'])

sorted_counter = sorted(counter.items())

sorted_counter

[(1, 21), (2, 41), (3, 103), (4, 226), (5, 609)]

In [ ]:
class Sentiment:
  NEGATIVE = 'NEGATIVE'
  NEUTRAL = 'NEUTRAL'
  POSITIVE = 'POSITIVE'

class Review:
  def __init__(self, text, score):
    self.text = text
    self.score = score

  def __repr__(self):
    return f'{self.text}, {self.score}'

  def get_sentiment(self):
    if self.score <= 2:
      return Sentiment.NEGATIVE
    elif self.score == 3:
      return Sentiment.NEUTRAL
    else:
      return Sentiment.POSITIVE


In [ ]:
data = []

for text, score in zip(df['reviewText'], df['overall']):
  data.append(Review(text, score))

data[1].get_sentiment()

'NEGATIVE'

In [ ]:
len(data)

1000

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.3, train_size=0.7, random_state=42)

len(train), len(test)

(700, 300)

In [ ]:
train_x = [x.text for x in train]
train_y = [x.get_sentiment() for x in train]

test_x = [x.text for x in test]
test_y = [x.get_sentiment() for x in test]

train_y[0]

'POSITIVE'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

train_x_vectors[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])


array(['POSITIVE'], dtype='<U8')

In [ ]:
clf_svm.score(test_x_vectors, test_y)

0.8233333333333334

In [ ]:
from sklearn.metrics import f1_score

f1_score_svm = f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE], zero_division=1)
f1_score_svm

array([0.9059501 , 0.26923077, 0.2962963 ])